# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [98]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [99]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [100]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    #return None, None
    vocab = set(text)
    vocab_to_int = {word: i for i, word in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [101]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    #return None
    punc_dict = {'.': '||Period||', ',': '||Comma||', '"': '||Quotation_Mark||', 
                ';': '||Semicolon||', '!': '||Exclamation_Mark||', '?': '||Question_Mark||', 
                '(': '||Left_Parentheses||', ')': '||Right_Parantheses||', '--': '||Dash||', 
                '\n': '||Return||'}
    return punc_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [102]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [103]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [104]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [105]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    #return None, None, None
    
    input = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return input, targets, learning_rate
    


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [106]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    #return None, None
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=0.8)
    cell = tf.contrib.rnn.MultiRNNCell([drop] * 5)

    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    
    return cell, initial_state



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [107]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    #return None
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [108]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    #return None, None
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs,
                                             dtype=tf.float32)
    
    final_state = tf.identity(final_state, name='final_state')
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [109]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    #return None, None
    #some modifcation in parameters
    embed_input = get_embed(input_data, vocab_size, embed_dim)
    #outputs, final_state = tf.nn.dynamic_rnn(cell, embed_input ,dtype=tf.float32)
    outputs, final_state = build_rnn(cell, embed_input)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [110]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    #return None  ydata[-1] = xdata[0]
    n_batch = len(int_text)//(batch_size*seq_length)
    x_data = np.array(int_text[:batch_size*seq_length*n_batch])
    y_data = np.array(int_text[1:batch_size*seq_length*n_batch+1])
    y_data[-1] = x_data[0]
    #y_batches[n_batches-1][batch_size-1][seq_length-1]
    x_batches = np.split(x_data.reshape(batch_size, -1), n_batch, 1)
    y_batches = np.split(y_data.reshape(batch_size, -1), n_batch, 1)
    

    return np.array(list(zip(x_batches, y_batches)))


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [111]:
"""
# Number of Epochs
num_epochs = 1000
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 256
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 50
#withou embed_dim we will get embed_dim is not defined error
embed_dim=128

"""
# Number of Epochs
num_epochs = 700
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 512
# Sequence Length
seq_length = 13
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 5

#embed_dim=128

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'


### Build the Graph
Build the graph using the neural network you implemented.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)
    #logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/10   train_loss = 8.822
Epoch   0 Batch    5/10   train_loss = 6.924
Epoch   1 Batch    0/10   train_loss = 6.846
Epoch   1 Batch    5/10   train_loss = 6.223
Epoch   2 Batch    0/10   train_loss = 6.345
Epoch   2 Batch    5/10   train_loss = 6.224
Epoch   3 Batch    0/10   train_loss = 6.137
Epoch   3 Batch    5/10   train_loss = 6.110
Epoch   4 Batch    0/10   train_loss = 6.143
Epoch   4 Batch    5/10   train_loss = 6.092
Epoch   5 Batch    0/10   train_loss = 6.057
Epoch   5 Batch    5/10   train_loss = 6.090
Epoch   6 Batch    0/10   train_loss = 6.080
Epoch   6 Batch    5/10   train_loss = 6.079
Epoch   7 Batch    0/10   train_loss = 6.060
Epoch   7 Batch    5/10   train_loss = 6.075
Epoch   8 Batch    0/10   train_loss = 6.061
Epoch   8 Batch    5/10   train_loss = 6.070
Epoch   9 Batch    0/10   train_loss = 6.058
Epoch   9 Batch    5/10   train_loss = 6.058
Epoch  10 Batch    0/10   train_loss = 6.053
Epoch  10 Batch    5/10   train_loss = 6.065
Epoch  11 

Epoch  91 Batch    5/10   train_loss = 5.794
Epoch  92 Batch    0/10   train_loss = 5.751
Epoch  92 Batch    5/10   train_loss = 5.795
Epoch  93 Batch    0/10   train_loss = 5.738
Epoch  93 Batch    5/10   train_loss = 5.792
Epoch  94 Batch    0/10   train_loss = 5.746
Epoch  94 Batch    5/10   train_loss = 5.783
Epoch  95 Batch    0/10   train_loss = 5.737
Epoch  95 Batch    5/10   train_loss = 5.770
Epoch  96 Batch    0/10   train_loss = 5.718
Epoch  96 Batch    5/10   train_loss = 5.775
Epoch  97 Batch    0/10   train_loss = 5.715
Epoch  97 Batch    5/10   train_loss = 5.767
Epoch  98 Batch    0/10   train_loss = 5.718
Epoch  98 Batch    5/10   train_loss = 5.758
Epoch  99 Batch    0/10   train_loss = 5.714
Epoch  99 Batch    5/10   train_loss = 5.766
Epoch 100 Batch    0/10   train_loss = 5.715
Epoch 100 Batch    5/10   train_loss = 5.764
Epoch 101 Batch    0/10   train_loss = 5.708
Epoch 101 Batch    5/10   train_loss = 5.754
Epoch 102 Batch    0/10   train_loss = 5.698
Epoch 102 

Epoch 183 Batch    0/10   train_loss = 3.925
Epoch 183 Batch    5/10   train_loss = 3.977
Epoch 184 Batch    0/10   train_loss = 3.892
Epoch 184 Batch    5/10   train_loss = 3.970
Epoch 185 Batch    0/10   train_loss = 3.876
Epoch 185 Batch    5/10   train_loss = 3.940
Epoch 186 Batch    0/10   train_loss = 3.867
Epoch 186 Batch    5/10   train_loss = 3.931
Epoch 187 Batch    0/10   train_loss = 3.855
Epoch 187 Batch    5/10   train_loss = 3.916
Epoch 188 Batch    0/10   train_loss = 3.855
Epoch 188 Batch    5/10   train_loss = 3.921
Epoch 189 Batch    0/10   train_loss = 3.845
Epoch 189 Batch    5/10   train_loss = 3.888
Epoch 190 Batch    0/10   train_loss = 3.807
Epoch 190 Batch    5/10   train_loss = 3.891
Epoch 191 Batch    0/10   train_loss = 3.812
Epoch 191 Batch    5/10   train_loss = 3.897
Epoch 192 Batch    0/10   train_loss = 3.797
Epoch 192 Batch    5/10   train_loss = 3.864
Epoch 193 Batch    0/10   train_loss = 3.757
Epoch 193 Batch    5/10   train_loss = 3.854
Epoch 194 

Epoch 274 Batch    5/10   train_loss = 3.086
Epoch 275 Batch    0/10   train_loss = 3.004
Epoch 275 Batch    5/10   train_loss = 3.067
Epoch 276 Batch    0/10   train_loss = 3.020
Epoch 276 Batch    5/10   train_loss = 3.067
Epoch 277 Batch    0/10   train_loss = 2.999
Epoch 277 Batch    5/10   train_loss = 3.071
Epoch 278 Batch    0/10   train_loss = 3.000
Epoch 278 Batch    5/10   train_loss = 3.063
Epoch 279 Batch    0/10   train_loss = 2.999
Epoch 279 Batch    5/10   train_loss = 3.055
Epoch 280 Batch    0/10   train_loss = 2.994
Epoch 280 Batch    5/10   train_loss = 3.056
Epoch 281 Batch    0/10   train_loss = 2.971
Epoch 281 Batch    5/10   train_loss = 3.049
Epoch 282 Batch    0/10   train_loss = 2.953
Epoch 282 Batch    5/10   train_loss = 3.025
Epoch 283 Batch    0/10   train_loss = 2.974
Epoch 283 Batch    5/10   train_loss = 3.042
Epoch 284 Batch    0/10   train_loss = 2.964
Epoch 284 Batch    5/10   train_loss = 3.032
Epoch 285 Batch    0/10   train_loss = 2.967
Epoch 285 

Epoch 366 Batch    0/10   train_loss = 2.577
Epoch 366 Batch    5/10   train_loss = 2.624
Epoch 367 Batch    0/10   train_loss = 2.583
Epoch 367 Batch    5/10   train_loss = 2.656
Epoch 368 Batch    0/10   train_loss = 2.613
Epoch 368 Batch    5/10   train_loss = 2.624
Epoch 369 Batch    0/10   train_loss = 2.591
Epoch 369 Batch    5/10   train_loss = 2.615
Epoch 370 Batch    0/10   train_loss = 2.561
Epoch 370 Batch    5/10   train_loss = 2.639
Epoch 371 Batch    0/10   train_loss = 2.574
Epoch 371 Batch    5/10   train_loss = 2.670
Epoch 372 Batch    0/10   train_loss = 2.569
Epoch 372 Batch    5/10   train_loss = 2.626
Epoch 373 Batch    0/10   train_loss = 2.605
Epoch 373 Batch    5/10   train_loss = 2.628
Epoch 374 Batch    0/10   train_loss = 2.551
Epoch 374 Batch    5/10   train_loss = 2.611
Epoch 375 Batch    0/10   train_loss = 2.570
Epoch 375 Batch    5/10   train_loss = 2.601
Epoch 376 Batch    0/10   train_loss = 2.551
Epoch 376 Batch    5/10   train_loss = 2.609
Epoch 377 

Epoch 457 Batch    5/10   train_loss = 2.378
Epoch 458 Batch    0/10   train_loss = 2.380
Epoch 458 Batch    5/10   train_loss = 2.354
Epoch 459 Batch    0/10   train_loss = 2.366
Epoch 459 Batch    5/10   train_loss = 2.399
Epoch 460 Batch    0/10   train_loss = 2.373
Epoch 460 Batch    5/10   train_loss = 2.395
Epoch 461 Batch    0/10   train_loss = 2.374
Epoch 461 Batch    5/10   train_loss = 2.368
Epoch 462 Batch    0/10   train_loss = 2.363
Epoch 462 Batch    5/10   train_loss = 2.400
Epoch 463 Batch    0/10   train_loss = 2.347
Epoch 463 Batch    5/10   train_loss = 2.391
Epoch 464 Batch    0/10   train_loss = 2.391
Epoch 464 Batch    5/10   train_loss = 2.381
Epoch 465 Batch    0/10   train_loss = 2.383
Epoch 465 Batch    5/10   train_loss = 2.385
Epoch 466 Batch    0/10   train_loss = 2.356
Epoch 466 Batch    5/10   train_loss = 2.371
Epoch 467 Batch    0/10   train_loss = 2.366
Epoch 467 Batch    5/10   train_loss = 2.380
Epoch 468 Batch    0/10   train_loss = 2.359
Epoch 468 

Epoch 550 Batch    0/10   train_loss = 2.299
Epoch 550 Batch    5/10   train_loss = 2.282
Epoch 551 Batch    0/10   train_loss = 2.295
Epoch 551 Batch    5/10   train_loss = 2.289
Epoch 552 Batch    0/10   train_loss = 2.299
Epoch 552 Batch    5/10   train_loss = 2.288
Epoch 553 Batch    0/10   train_loss = 2.259
Epoch 553 Batch    5/10   train_loss = 2.298
Epoch 554 Batch    0/10   train_loss = 2.269
Epoch 554 Batch    5/10   train_loss = 2.302
Epoch 555 Batch    0/10   train_loss = 2.307
Epoch 555 Batch    5/10   train_loss = 2.279
Epoch 556 Batch    0/10   train_loss = 2.261
Epoch 556 Batch    5/10   train_loss = 2.265
Epoch 557 Batch    0/10   train_loss = 2.268
Epoch 557 Batch    5/10   train_loss = 2.249
Epoch 558 Batch    0/10   train_loss = 2.256
Epoch 558 Batch    5/10   train_loss = 2.289
Epoch 559 Batch    0/10   train_loss = 2.284
Epoch 559 Batch    5/10   train_loss = 2.261
Epoch 560 Batch    0/10   train_loss = 2.256
Epoch 560 Batch    5/10   train_loss = 2.270
Epoch 561 

Epoch 641 Batch    5/10   train_loss = 2.200
Epoch 642 Batch    0/10   train_loss = 2.221
Epoch 642 Batch    5/10   train_loss = 2.182
Epoch 643 Batch    0/10   train_loss = 2.247
Epoch 643 Batch    5/10   train_loss = 2.241
Epoch 644 Batch    0/10   train_loss = 2.240
Epoch 644 Batch    5/10   train_loss = 2.216
Epoch 645 Batch    0/10   train_loss = 2.226
Epoch 645 Batch    5/10   train_loss = 2.188
Epoch 646 Batch    0/10   train_loss = 2.228
Epoch 646 Batch    5/10   train_loss = 2.223
Epoch 647 Batch    0/10   train_loss = 2.224
Epoch 647 Batch    5/10   train_loss = 2.230
Epoch 648 Batch    0/10   train_loss = 2.252
Epoch 648 Batch    5/10   train_loss = 2.263
Epoch 649 Batch    0/10   train_loss = 2.251
Epoch 649 Batch    5/10   train_loss = 2.254
Epoch 650 Batch    0/10   train_loss = 2.266
Epoch 650 Batch    5/10   train_loss = 2.249
Epoch 651 Batch    0/10   train_loss = 2.239
Epoch 651 Batch    5/10   train_loss = 2.236
Epoch 652 Batch    0/10   train_loss = 2.242
Epoch 652 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [ ]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    #return None, None, None, None
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [ ]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    #return None
    return int_to_vocab[np.argmax(probabilities)]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [ ]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:
hank_williams_jr(german) i can't be" be be no to get the boat.
moe_szyslak:(to snort) lousy on a last song at old pick.
homer_simpson:(very ralph) quiet grab what.
moe_szyslak:(clears him) hey, ha ain't a most front.
moe_szyslak: ol' a ad to go, glove me?.. fifty out.


moe_szyslak:(" self-satisfied fly the" early) i just put that cologne.
barney_gumble: okay, i nation to she mention light.


moe_szyslak:(sweetly) wow, god, don't stole i'd ain't go.
c. _montgomery_burns: you already me to shot.
moe_szyslak:(finishing accent) oh ned, i doin' know much arm.
homer_simpson:(counting out springfield) woo!
moe_szyslak:(offended) you got jerry back a bon-bons an ugly.
homer_simpson:(thoughtful) hey, homer.
homer_simpson:(cheery) i'll were prank from you tell a ideas who's.
lisa_simpson: that'll like to go.
moe_szyslak:(


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.